<a href="https://colab.research.google.com/github/vinhhiep96/06-House_Prices_Advanced_Regression-Techniques-Sergei_Neviadomski/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_B%E1%BA%A3n_sao_c%E1%BB%A7a_B%E1%BA%A3n_sao_c%E1%BB%A7a_23_09_Airline_Dataset_kaggle_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import library

In [1]:
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
warnings.filterwarnings('ignore')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

sh=gc.open_by_url('https://docs.google.com/spreadsheets/d/1O0mIfMV1qNZlHlmOnZtdnvSPJ16U_A1CzMePi3MelaM/edit?usp=drivesdk')
df = pd.DataFrame(sh.get_worksheet(0).get_all_records())
df.columns=df.columns.str.replace(' ', '_').str.lower().to_list()

In [2]:
# Unique Values of Each Feature

In [3]:
def def1(df):
  l1=[]
  l2=[]
  for i in df.columns:
    A=df[i].nunique()
    l1.append(A)
    B=df[i].unique()[:5]
    l2.append(B)
  return pd.DataFrame({'number_unique':l1, 'unique': l2},index=df.columns)
df1=def1(df)
df1

,number_unique,unique
booking_id,36275,"[INN00001, INN00002, INN00003, INN00004, INN00..."
no_of_adults,5,"[2, 1, 3, 0, 4]"
no_of_children,6,"[0, 2, 1, 3, 10]"
no_of_weekend_nights,8,"[1, 2, 0, 4, 3]"
no_of_week_nights,18,"[2, 3, 1, 4, 5]"
type_of_meal_plan,4,"[Meal Plan 1, Not Selected, Meal Plan 2, Meal ..."
required_car_parking_space,2,"[0, 1]"
room_type_reserved,7,"[Room_Type 1, Room_Type 4, Room_Type 2, Room_T..."
lead_time,352,"[224, 5, 1, 211, 48]"
arrival_year,2,"[2017, 2018]"


In [4]:
# Percentage of Canceled Orders

In [5]:
l1=df.select_dtypes(include=['int']).columns
df1=df.groupby(['booking_status'])[l1].apply(lambda x: x.sum()*100/df[l1].sum())
df1=df1.T
fig=go.Figure()
fig=go.Figure()
fig.add_trace(go.Bar(x=df1.index,y=df1.Canceled,name='Canceled'))
fig.add_trace(go.Bar(x=df1.index,y=df1.Not_Canceled,name='Not_Canceled'))
fig=fig.update_layout(title="The percentage of canceled orders", yaxis_title="Percentage") #Added xaxis title, assuming that is what was intended
fig.show()

In [ ]:
# Number value of each feature

In [6]:
for i in df.columns:
  if df[i].dtype in [np.int64, np.float64]:
    l1 = [i]
    l2 = [df[i].mean()]
  elif type(df[i].value_counts().index) == pd.Index:
    if i == 'booking_id':
      continue
    l1 = df[i].value_counts().index.tolist()
    l2 = df[i].value_counts().values.tolist()
    print(l2)
    fig=px.bar(x=l1,y=l2, labels={'x':i, 'y': 'Couunt'})
  else:
    pass
  fig.show()

[27835, 5130, 3305, 5]


[28130, 6057, 966, 692, 265, 158, 7]


[23214, 10528, 2017, 391, 125]


[24390, 11885]


In [7]:
# Number value of Each Feature by Month

In [8]:
def def1(B):
  for i in df['arrival_year'].unique():
    fig = go.Figure()
    for m in range(df[df.arrival_year == i]['arrival_month'].value_counts().sort_index().index[0], 13):  # Call value_counts() and sort_index()
      df1 = df[df.arrival_month == m][B].value_counts().sort_index()
      fig.add_trace(go.Scatter(x=df1.index, y=df1.values, name = f'month {m}'))
      fig.update_layout(xaxis_title=B, yaxis_title='Count')
      fig.show()
def1('no_of_adults')

In [9]:
# Count of Average Price

In [10]:
l1=[]
df1=df['avg_price_per_room']
for i in range(df.shape[0]): # Use range to generate sequence of numbers
  if len(str(df['avg_price_per_room'][i]).split('.')[0])==2: # Added closing parenthesis for len()
    l1.append(str(df.loc[i, 'avg_price_per_room'])[:2]) # Accessing the value at row i and column 'avg_price_per_room', then slicing the string
  elif len(str(df['avg_price_per_room'][i]).split('.')[0])==3: # Added closing parenthesis for len()
    l1.append(str(df.loc[i, 'avg_price_per_room'])[:3]) # Accessing the value at row i and column 'avg_price_per_room', then slicing the string
  else:
    l1.append(0) # Append NaN for cases where integer part has more than 2 digits

# Create a new Series from l1 and give it a name, ensuring it has the same index as df1
new_series = pd.Series(l1, name='avg_price_per_room_map', index=df1.index)

# Concatenate df1 (original Series) with the new Series along columns (axis=1)
df1 = pd.concat([df1, new_series], axis=1)
df1.avg_price_per_room_map=df1.avg_price_per_room_map.astype(int)
df2=df1['avg_price_per_room_map'].value_counts().sort_index()
fig=px.bar(df2)
fig.show()
#df1

In [11]:
# Number of Non-Repeated Guests/Repeated Guests

In [12]:
l1=[]
for i in range(df.shape[0]): # Use range to generate sequence of numbers
  if len(str(df['avg_price_per_room'][i]).split('.')[0])==2: # Added closing parenthesis for len()
    l1.append(str(df.loc[i, 'avg_price_per_room'])[:2]) # Accessing the value at row i and column 'avg_price_per_room', then slicing the string
  elif len(str(df['avg_price_per_room'][i]).split('.')[0])==3: # Added closing parenthesis for len()
    l1.append(str(df.loc[i, 'avg_price_per_room'])[:3]) # Accessing the value at row i and column 'avg_price_per_room', then slicing the string
  else:
    l1.append(0) # Append NaN for cases where integer part has more than 2 digits

# Create a new Series from l1 and give it a name, ensuring it has the same index as df1
new_series = pd.Series(l1, name='avg_price_per_room_map', index=df1.index)

# Concatenate df1 (original Series) with the new Series along columns (axis=1)
df1 = pd.concat([df, new_series], axis=1)

df4=pd.DataFrame()
df2=df1['avg_price_per_room_map'].astype(int).value_counts().sort_index().values
df5=df1['avg_price_per_room_map'].astype(int).value_counts().sort_index().index
df3=df1[df1['repeated_guest']==0]['avg_price_per_room_map'].astype(int).value_counts().sort_index()
l3=[df2,df3]
l2=['num guest','num guest not repeat']
for i, m in zip(l2,l3):
   df4.insert(0,i,m)
df4.insert(0,'index',df5)
df4.head(3)
fig=px.bar(df4, x='index', y=['num guest not repeat', 'num guest'])
fig.show()

In [13]:
# Booking status by market segment type

In [14]:
fig=make_subplots(rows=2, cols=1, specs=[[{'type':'pie'}],[ {'type':'pie'}]], subplot_titles=['online','offline'])
l1=[df[df['market_segment_type']=='Online'],df[df['market_segment_type']=='Offline']]
for m,n in enumerate(l1):
  df1=n.booking_status.value_counts().sort_index()
  fig.add_trace(go.Pie(labels=df1.index, values=df1.values, name=l1[m]['market_segment_type'].unique()[0]), row=m+1, col=1) # extract unique segment type for the name
fig.show()

In [16]:
# Average price by car parking space

In [18]:
A = 'non required_car_parking_space'
B = 'required_car_parking_space'
C='avg_price_per_room'
l1 = [df.loc[df[C] == 0, C].mean(), df.loc[df[C] == 1, C].mean()]
l2 = [A, B]
fig = go.Figure()
for i in range(len(l2)):
  fig.add_trace(go.Bar(x=[l2[i]], y=[l1[i]], name=l2[i])) #x should be a list of strings, y should be a list of values
fig.show()